In [ ]:
%load_ext autoreload
%autoreload 2

import csv
import json
import os
import pandas as pd
import papermill as pm
import re
import scrapbook as sb
import uuid

from functions import gpt

from tqdm.notebook import tqdm_notebook
tqdm_notebook().pandas()

pd.set_option('display.max_colwidth', None)

from pylatexenc.latexwalker import LatexWalker, LatexMathNode, LatexMacroNode, LatexGroupNode, LatexCharsNode, LatexEnvironmentNode

In [ ]:
base_name = "2021_Hashimoto_Neural_ODE_and_holographic_QCD_PUB"
project_folder = "diygenomics-projects"
sub_category = "math"
work_bucket = "AdS-CFT"
external_id = "2023_05_22_92dc0613b4493d7b5847g"

In [ ]:
data_path = os.getenv('DATA_PATH')
file_path = lambda *args: os.path.join(data_path, project_folder, sub_category, work_bucket,
                                       base_name, 'mathpix', *args)

model = 'gpt-4' # 'gpt-3.5-turbo' # 'gpt-4'
index_col = 'uuid'

laytex_file = f'{external_id}.tex'

In [ ]:
with open(file_path(external_id, laytex_file), 'r') as f:
    latex_code = f.read()
walker = LatexWalker(latex_code)

In [ ]:
inline_math = re.findall(r'\$([^$]*)\$', latex_code)
displayed_math = re.findall(r'\\\[(.*?)\\\]', latex_code, re.DOTALL)
equation_math = re.findall(r'\\begin{equation}(.*?)\\end{equation}', latex_code, re.DOTALL)  

In [ ]:
print(len(inline_math))
print(len(displayed_math))
print(len(equation_math))

In [ ]:
data = {'math': inline_math + displayed_math + equation_math,
        'type': ['inline'] * len(inline_math) + ['display'] * len(displayed_math) + ['equation'] * len(equation_math)}

uuids = [uuid.uuid4() for _ in range(len(data['math']))]

df = pd.DataFrame(data, index=uuids)
df = df.rename_axis('uuid', axis='index')

In [ ]:
df.to_csv(file_path('extracted_math.csv'), quoting=csv.QUOTE_MINIMAL)

In [ ]:
# parsed_inline_math = []
# parsed_displayed_math = []

# for node in walker.get_latex_nodes():
#     if isinstance(node, list):
#         for sub_node in node:
#             if isinstance(sub_node, LatexEnvironmentNode): # LatexGroupNode LatexEnvironmentNode LatexMacroNode LatexCharsNode
#                 children = sub_node.nodelist
#                 for child_node in children:
#                     if isinstance(child_node, LatexMathNode):
#                         math_nodes = child_node.nodelist
#                         for math_node in math_nodes:
#                             print(math_node)
#                             # if isinstance(char_node, LatexCharsNode):
#                             #     print(char_node)
#                         if child_node.displaytype == 'inline':
#                             parsed_inline_math.append('a')
#                         elif child_node.displaytype == 'display':
#                             parsed_displayed_math.append('b')
#                         # nodelist

In [ ]:
# print(len(parsed_inline_math))
# print(len(parsed_displayed_math))